```
--- Day 18: RAM Run ---
You and The Historians look a lot more pixelated than you remember. You're inside a computer at the North Pole!

Just as you're about to check out your surroundings, a program runs up to you. "This region of memory isn't safe! The User misunderstood what a pushdown automaton is and their algorithm is pushing whole bytes down on top of us! Run!"

The algorithm is fast - it's going to cause a byte to fall into your memory space once every nanosecond! Fortunately, you're faster, and by quickly scanning the algorithm, you create a list of which bytes will fall (your puzzle input) in the order they'll land in your memory space.

Your memory space is a two-dimensional grid with coordinates that range from 0 to 70 both horizontally and vertically. However, for the sake of example, suppose you're on a smaller grid with coordinates that range from 0 to 6 and the following list of incoming byte positions:

5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
Each byte position is given as an X,Y coordinate, where X is the distance from the left edge of your memory space and Y is the distance from the top edge of your memory space.

You and The Historians are currently in the top left corner of the memory space (at 0,0) and need to reach the exit in the bottom right corner (at 70,70 in your memory space, but at 6,6 in this example). You'll need to simulate the falling bytes to plan out where it will be safe to run; for now, simulate just the first few bytes falling into your memory space.

As bytes fall into your memory space, they make that coordinate corrupted. Corrupted memory coordinates cannot be entered by you or The Historians, so you'll need to plan your route carefully. You also cannot leave the boundaries of the memory space; your only hope is to reach the exit.

In the above example, if you were to draw the memory space after the first 12 bytes have fallen (using . for safe and # for corrupted), it would look like this:

...#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#....
You can take steps up, down, left, or right. After just 12 bytes have corrupted locations in your memory space, the shortest path from the top left corner to the exit would take 22 steps. Here (marked with O) is one such path:

OO.#OOO
.O#OO#O
.OOO#OO
...#OO#
..#OO#.
.#.O#..
#.#OOOO
Simulate the first kilobyte (1024 bytes) falling onto your memory space. Afterward, what is the minimum number of steps needed to reach the exit?
```

In [4]:
from collections import deque

def read_input(file_path):
    with open(file_path, "r") as file:
        return [tuple(map(int, line.strip().split(','))) for line in file]

def simulate_corruption(grid, corrupted_positions):
    for x, y in corrupted_positions:
        grid[y][x] = "#"

def bfs_shortest_path(grid, start, goal):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
    queue = deque([(start[0], start[1], 0)])  # (x, y, steps)
    visited = set()
    visited.add(start)

    while queue:
        x, y, steps = queue.popleft()

        if (x, y) == goal:
            return steps

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 71 and 0 <= ny < 71 and (nx, ny) not in visited and grid[ny][nx] == ".":
                visited.add((nx, ny))
                queue.append((nx, ny, steps + 1))
    return -1  # If no path exists

def main():
    # Initialize the grid
    grid_size = 71
    grid = [["."] * grid_size for _ in range(grid_size)]

    # Read input and simulate falling bytes
    corrupted_positions = read_input("input.txt")[:1024]
    simulate_corruption(grid, corrupted_positions)

    # Find shortest path
    start = (0, 0)
    goal = (70, 70)
    result = bfs_shortest_path(grid, start, goal)

    print("Minimum number of steps:", result)

if __name__ == "__main__":
    main()


Minimum number of steps: 404


```
--- Part Two ---
The Historians aren't as used to moving around in this pixelated universe as you are. You're afraid they're not going to be fast enough to make it to the exit before the path is completely blocked.

To determine how fast everyone needs to go, you need to determine the first byte that will cut off the path to the exit.

In the above example, after the byte at 1,1 falls, there is still a path to the exit:

O..#OOO
O##OO#O
O#OO#OO
OOO#OO#
###OO##
.##O###
#.#OOOO
However, after adding the very next byte (at 6,1), there is no longer a path to the exit:

...#...
.##..##
.#..#..
...#..#
###..##
.##.###
#.#....
So, in this example, the coordinates of the first byte that prevents the exit from being reachable are 6,1.

Simulate more of the bytes that are about to corrupt your memory space. What are the coordinates of the first byte that will prevent the exit from being reachable from your starting position? (Provide the answer as two integers separated by a comma with no other characters.)
```

In [5]:
import heapq

def find_first_blocking_byte(input_file):
    # Read coordinates of falling bytes
    with open(input_file, 'r') as f:
        coordinates = [tuple(map(int, line.strip().split(','))) for line in f]

    # Define grid size and movement directions
    GRID_SIZE = 70
    DIRECTIONS = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # Check if a path exists from start to goal
    def is_path_exists(corrupted):
        start = (0, 0)
        goal = (GRID_SIZE, GRID_SIZE)

        visited = set()
        queue = [start]
        visited.add(start)

        while queue:
            current = queue.pop(0)

            if current == goal:
                return True

            for dx, dy in DIRECTIONS:
                neighbor = (current[0] + dx, current[1] + dy)

                # Check if neighbor is valid (within grid and not corrupted)
                if (0 <= neighbor[0] <= GRID_SIZE and
                    0 <= neighbor[1] <= GRID_SIZE and
                    neighbor not in corrupted and
                    neighbor not in visited):
                    queue.append(neighbor)
                    visited.add(neighbor)

        return False

    # Track corrupted spaces and find first blocking byte
    corrupted = set()
    for i, coord in enumerate(coordinates):
        corrupted.add(coord)

        # Check if this coordinate blocks the path
        if not is_path_exists(corrupted):
            return coord

    return None  # No blocking byte found

# Read input from file and solve
result = find_first_blocking_byte('input.txt')
print(f"First blocking byte coordinates: {result[0]},{result[1]}")

First blocking byte coordinates: 27,60
